In [4]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn.metrics import roc_auc_score, log_loss, accuracy_score

In [5]:
train = pd.read_csv('../data/train_processed.csv')
test = pd.read_csv('../data/test_processed.csv')

In [6]:
train.head()

,clickTime,positionID,creativeID_1040,creativeID_1052,creativeID_1064,creativeID_1069,creativeID_1073,creativeID_1086,creativeID_1087,creativeID_1092,...,userID_999953,userID_999969,connectionType_1,connectionType_2,connectionType_3,connectionType_4,telecomsOperator_1,telecomsOperator_2,telecomsOperator_3,label
0,0.0,0.035944,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
1,0.0,0.805720,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0
2,0.0,0.972714,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,0
3,0.0,0.125672,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
4,0.0,0.481307,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0


In [8]:
train_label = train['label']
train.drop('label', axis=1, inplace=True)

test_instanceID = test['instanceID']
test.drop(['label', 'instanceID'], axis=1, inplace=True)

In [9]:
train.head()

,clickTime,positionID,creativeID_1040,creativeID_1052,creativeID_1064,creativeID_1069,creativeID_1073,creativeID_1086,creativeID_1087,creativeID_1092,...,userID_999852,userID_999953,userID_999969,connectionType_1,connectionType_2,connectionType_3,connectionType_4,telecomsOperator_1,telecomsOperator_2,telecomsOperator_3
0,0.0,0.035944,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
1,0.0,0.805720,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
2,0.0,0.972714,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,0,0
3,0.0,0.125672,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
4,0.0,0.481307,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0


In [10]:
# 划分训练集和验证集
x_train, x_val, y_train, y_val = train_test_split(train, train_label, random_state=1)

In [11]:
# 逻辑回归，详细参数查看sklearn官网介绍，http://scikit-learn.org/stable/
logit = LogisticRegression(penalty='l2', solver='lbfgs', tol=0.01)

# 训练模型
logit.fit(x_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='lbfgs', tol=0.01,
          verbose=0, warm_start=False)

In [12]:
# 打印学习到的模型参数
print(logit.coef_)

[[ 0.12832984 -0.17581712  0.         ...,  0.38229435  0.27494334
   0.35595532]]


In [14]:
# 对测试集进行预测
# pred = logit.predict_proba(x_test)  # 输出为0和为1的概率
pred_val = logit.predict(x_val) # 输出分类结果

In [15]:
pred_val

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [17]:
# 使用accuracy， auc, logloss等度量方法衡量模型性能
auc = roc_auc_score(y_val, pred_val)
accuracy = accuracy_score(y_val, pred_val)
logloss = log_loss(y_val, pred_val)
print('auc:', auc, 'logloss:', logloss)

auc: 0.5 logloss: 1.1190563552


In [18]:
# 对测试集进行预测
pred_test = logit.predict_proba(test)

In [20]:
pred_test[:, 1]

array([ 1.,  1.,  1., ...,  1.,  1.,  1.])

In [30]:
# 将预测为1的概率写入文件
sub = pd.DataFrame(test_instanceID)
sub['prob'] = pred_test[:,1]

In [31]:
sub.head()

,instanceID,prob
0,1,1.0
1,2,1.0
2,3,1.0
3,4,1.0
4,5,1.0


In [29]:
sub.to_csv('../output/submission.csv', index=False)